<h1>Segmenting and Clustering Neighborhoods in Toronto </H1>

Import all necessary Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from pandas.io.json import json_normalize
import requests

# set displaying all values from pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#import geocoder # import geocoder

In [4]:
# Address to latitude and longitude
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

In [5]:
# displaying map
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [6]:
# for Kmean clustering
from sklearn.cluster import KMeans

Download data from wikipage

In [7]:
!wget -q -O 'toronto_data.csv' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


Load and explore data

Drop cells with a borough that is Not assigned

In [8]:
df_toronto = pd.read_html('toronto_data.csv')[0]
df_toronto = df_toronto[df_toronto['Borough'] !='Not assigned']
df_toronto

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [9]:
df_toronto_groupby = df_toronto.groupby(['Postcode','Borough'],as_index=False)
df_toronto_groupby.first()


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1C,Scarborough,Highland Creek
2,M1E,Scarborough,Guildwood
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,East Birchmount Park
7,M1L,Scarborough,Clairlea
8,M1M,Scarborough,Cliffcrest
9,M1N,Scarborough,Birch Cliff


Combine postal code when multiple postal code a

In [10]:
df_toronto_groupby = df_toronto_groupby.agg(lambda x : ','.join(x))


In [11]:
df_toronto_groupby.loc[df_toronto_groupby['Neighbourhood']=='Not assigned','Neighbourhood']=df_toronto_groupby[df_toronto_groupby['Neighbourhood']=='Not assigned']['Borough']

In [12]:
df_toronto_groupby.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df_toronto_groupby.loc[df_toronto_groupby['Neighbourhood']==df_toronto_groupby['Borough']]

,Postcode,Borough,Neighbourhood


In [14]:
df_toronto_groupby.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
#Download cordinate data
!wget -q -O 'cordinate_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [16]:
# Read Cordinate data
df_cordinate = pd.read_csv('cordinate_data.csv')
df_cordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
# Merge both dataframe and append cordinates to each post code
df_toronto_cordinate = pd.merge(df_toronto_groupby,df_cordinate,left_on='Postcode',right_on='Postal Code',how='inner') 
df_toronto_cordinate

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848


In [23]:
# Get only records with toronto borough
df_toronto_borough = df_toronto_cordinate[df_toronto_cordinate['Borough'].str.find('Toronto')!=-1].reset_index(drop=True)
df_toronto_borough

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",M4K,43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",M4L,43.668999,-79.315572
3,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,M4P,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,M4R,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,M4S,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",M4T,43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",M4V,43.686412,-79.400049


In [27]:
# Get Cordinates for Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto_borough['Latitude'], df_toronto_borough['Longitude'], df_toronto_borough['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [29]:
# set credential for 
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0C5RF20Z3DEQPEKCURMKBYXNRXE2Q20TV0E4WYZRGPGLUO4E
CLIENT_SECRET:LAX2X2AGVTS4WL2XGF1X135UBVHDPJQKHZYQX5QO03OG2ISS
